In [1]:
import pandas as pd
import cx_Oracle
import pyodbc
import time
from datetime import datetime
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.types import String
from config import oracle_wfm_key
from config import oracle_ods_key
from tqdm import tqdm
import itertools
import databases as db 

verint = db.ssms()
ods = db.oracle('cco_wfm',oracle_wfm_key)


In [2]:
ee = '''select 
            EmployeeName,
            EmployeeID as employeeID,
            City, 
            StateName as State,
            Country
        from V_AdHoc_EmployeeContacts
        where (USERNAME = 'satverintfirstlogon'
            AND ContactMethod = 'PopupAddress')'''
ee_df = pd.read_sql(ee,verint)

In [3]:
current = dt.date.today()
start = dt.date.today() - dt.timedelta(days=60)
start_date = start
end_date = current

In [4]:
coaching = f'''SELECT DISTINCT 
                        workDayLocal as Date,
                        startTimeLocal as Start_Time,
                        endTimeLocal as End_Time,
                        organizationName,
                        employeeID,
                        activityName as Absence,
                        Duration = DATEDIFF(second, startTimeLocal, endTimeLocal)/3600.0
                FROM V_RPT_PLANNEDACTIVITIES
                WHERE (isPaid = 1) 
                    AND (isUnPublished = 0) 
                    AND ((activityIsAbsence = 1) OR (activityIsCalendar = 1))
                    AND ((mediaName IS NULL)
                        OR (mediaName = 'Phone' AND activityName LIKE '%Phone')
                        OR (mediaName = 'Chat' AND activityName LIKE '%Chat%')
                        OR (mediaName = 'Project')
                        OR (mediaName = 'Phone Outbound' AND activityName = 'Outbound')
                        OR (mediaName = 'Face-to-Face' AND activityName <> 'Blended'))
                    AND (activityName LIKE '%Coach%')
                    AND (workDayLocal BETWEEN '{start_date}' AND '{end_date}')'''
coach_db = pd.read_sql(coaching,verint)
coach_db.head(3)

,Date,Start_Time,End_Time,organizationName,employeeID,Absence,Duration
0,2023-05-26,2023-05-26 07:00:00,2023-05-26 09:36:00,Resi Wireline Repair,32100,Coaching,2.600000
1,2023-05-26,2023-05-26 07:53:00,2023-05-26 08:28:00,Resi Wireline Repair,42254,Coaching,0.583333
2,2023-05-26,2023-05-26 08:24:00,2023-05-26 09:24:00,Resi Wireline Repair,42227,QCC Coaching BPO,1.000000


In [5]:
lmonitor = f'''SELECT DISTINCT 
                        workDayLocal as Date,
                        startTimeLocal as Start_Time,
                        endTimeLocal as End_Time,
                        organizationName,
                        employeeID,
                        activityName as Absence,
                        Duration = DATEDIFF(second, startTimeLocal, endTimeLocal)/3600.0
                FROM V_RPT_PLANNEDACTIVITIES
                WHERE ([isPaid] = 1) 
                    AND ([isUnPublished] = 0) 
                    AND (([mediaName] IS NULL)
                        OR ([mediaName] = 'Phone')
                        OR ([mediaName] = 'Chat' AND activityName LIKE '%Chat%')
                        OR ([mediaName] = 'Project')
                        OR ([mediaName] = 'Phone Outbound' AND activityName = 'Outbound')
                        OR ([mediaName] = 'Face-to-Face' AND activityName <> 'Blended'))
                    AND ([activityName] LIKE '%Live%')
                    AND (workDayLocal BETWEEN '{start_date}' AND '{end_date}')'''
lm_db = pd.read_sql(lmonitor,verint)
lm_db.head(3)

,Date,Start_Time,End_Time,organizationName,employeeID,Absence,Duration
0,2023-05-26,2023-05-26 09:00:00,2023-05-26 10:30:00,Cable Sales Retail,32313,Supervisor Live Monitor,1.50
1,2023-05-26,2023-05-26 09:00:00,2023-05-26 10:45:00,Cable Sales Retail,35601,Supervisor Live Monitor,1.75
2,2023-05-26,2023-05-26 09:45:00,2023-05-26 10:45:00,ITEL Wireline Consumer Specialists,30019,Live Monitor,1.00


In [6]:
activities = coach_db.merge(ee_df, on='employeeID',how='left')
activities = pd.concat([activities,lm_db])
activities.head(3)

,Date,Start_Time,End_Time,organizationName,employeeID,Absence,Duration,EmployeeName,City,State,Country
0,2023-05-26,2023-05-26 07:00:00,2023-05-26 09:36:00,Resi Wireline Repair,32100,Coaching,2.600000,"Whyte, Shavelle",None,None,None
1,2023-05-26,2023-05-26 07:53:00,2023-05-26 08:28:00,Resi Wireline Repair,42254,Coaching,0.583333,"Solomon-Clarke, Ashley",None,None,None
2,2023-05-26,2023-05-26 08:24:00,2023-05-26 09:24:00,Resi Wireline Repair,42227,QCC Coaching BPO,1.000000,"Bernard, Faraji",None,None,None


In [7]:
ods = db.oracle('usrocu_support',oracle_ods_key)
location = f'''select
                "A"."AGENT_USER_ID" "usr",
                "A"."LOCATION"
            from "CCO_OBJECTS"."BPO_USER_LOCATION" "A"'''
location_df = pd.read_sql(location,ods)
location_df['usr'] = location_df['usr'].str.lower()
usr = f'''select distinct EmployeeID as employeeID,
                ExternalEmpIdent as usr
        from V_AdHoc_EmployeeDataSource
        where DataSourceID = 1'''
usr_df = pd.read_sql(usr,verint)
usr_df['usr'] = usr_df['usr'].str.lower()

In [8]:
activity_usr = activities.merge(usr_df, on='employeeID',how='left')
activity_usr.head(3)

,Date,Start_Time,End_Time,organizationName,employeeID,Absence,Duration,EmployeeName,City,State,Country,usr
0,2023-05-26,2023-05-26 07:00:00,2023-05-26 09:36:00,Resi Wireline Repair,32100,Coaching,2.600000,"Whyte, Shavelle",None,None,None,swhyt1
1,2023-05-26,2023-05-26 07:53:00,2023-05-26 08:28:00,Resi Wireline Repair,42254,Coaching,0.583333,"Solomon-Clarke, Ashley",None,None,None,asolo1
2,2023-05-26,2023-05-26 08:24:00,2023-05-26 09:24:00,Resi Wireline Repair,42227,QCC Coaching BPO,1.000000,"Bernard, Faraji",None,None,None,fbern1


In [9]:
table = activity_usr.merge(location_df,on='usr',how='left')
table['month'] = table['Date'].dt.to_period('M').dt.to_timestamp()
table.head(3)

,Date,Start_Time,End_Time,organizationName,employeeID,Absence,Duration,EmployeeName,City,State,Country,usr,location,month
0,2023-05-26,2023-05-26 07:00:00,2023-05-26 09:36:00,Resi Wireline Repair,32100,Coaching,2.600000,"Whyte, Shavelle",None,None,None,swhyt1,WAH MBJ,2023-05-01
1,2023-05-26,2023-05-26 07:53:00,2023-05-26 08:28:00,Resi Wireline Repair,42254,Coaching,0.583333,"Solomon-Clarke, Ashley",None,None,None,asolo1,OFFICE MBJ,2023-05-01
2,2023-05-26,2023-05-26 08:24:00,2023-05-26 09:24:00,Resi Wireline Repair,42227,QCC Coaching BPO,1.000000,"Bernard, Faraji",None,None,None,fbern1,OFFICE MBJ,2023-05-01


In [10]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
delete = f'''delete from "CCO_WFM"."Coaching_Activities" where "Date" >= TO_DATE('{start.strftime('%Y-%m-%d')}', 'YYYY-MM-DD')'''
drop =f'drop table "CCO_WFM"."Coaching_Activities"'
with ods as conn:
    conn.execute(drop)

In [11]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
cols = table.dtypes[table.dtypes=='object'].index
type_mapping = {col : String(100) for col in cols }
table.to_sql('Coaching_Activities',con=ods,schema='CCO_WFM',if_exists='append',index=False,chunksize=10**4,dtype=type_mapping)

7117

In [12]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7117 entries, 0 to 7116
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              7117 non-null   datetime64[ns]
 1   Start_Time        7117 non-null   datetime64[ns]
 2   End_Time          7117 non-null   datetime64[ns]
 3   organizationName  7117 non-null   object        
 4   employeeID        7117 non-null   int64         
 5   Absence           7117 non-null   object        
 6   Duration          7117 non-null   float64       
 7   EmployeeName      5721 non-null   object        
 8   City              3496 non-null   object        
 9   State             587 non-null    object        
 10  Country           3577 non-null   object        
 11  usr               7117 non-null   object        
 12  location          6932 non-null   object        
 13  month             7117 non-null   datetime64[ns]
dtypes: datetime64[ns](4), fl

In [13]:
table.loc[(table['EmployeeName'] == 'Barnett, Mikeat')]['Duration'].sum()

5.166666